In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output
from ipywidgets import *
import markdown

In [2]:
sys.path.append("apollo")
from apollo.agent import Agent
agent = None

# Apollo Dashboard

## Create a New Experiment

#### To create a new experiment, first create a CSV file that contains the following columns

- **SMILES:** The cannonical SMILES string for a given compound
- **Score:** The efficacy score for a given compound where a score of zero is inert and a higher score corresponds to a more effective compound (Efficacy scores should be no greater than 10 for optimal performance).
- Any other identifying information

#### An example of such a file is shown below

|Name|SMILES|Score|
|---|---|---|
|Dopamine|C1=CC(=C(C=C1CCN)O)O|5.2|
|Serotonin|C1=CC2=C(C=C1O)C(=CN2)CCN|3.4|
|Epinephrine|CNCC(C1=CC(=C(C=C1)O)O)O|7.1|
|Norepinephrine|C1=CC(=C(C=C1C(CN)O)O)O|4.6|

#### Once you've filled out the file, fill out the following form and click Create Experiment

In [3]:
form_layout = Layout(display='flex',flex_flow='column',justify_content='space-between',width="100%")
form_row_layout = Layout(display='flex',flex_flow='row',justify_content='space-between',height="auto",width="100%")
form_item_layout = Layout(display='flex',flex_flow='column',justify_content='space-between',flex='1')
large_form_item_layout = Layout(display='flex',flex_flow='column',height="auto",justify_content='space-between',flex='3')
form_input_layout = Layout(width="auto")
form_button_layout = Layout(width="50%",margin="10px auto")

name = Text(layout=form_input_layout)
lr = FloatText(layout=form_input_layout, value=0.01)
weight_decay = FloatText(layout=form_input_layout, value=1e-05)
dropout = FloatText(layout=form_input_layout, value=0.25)
leaky_relu_coefficient = FloatText(layout=form_input_layout,value=0.01)
num_blocks = IntText(layout=form_input_layout, value=4)
num_channels = IntText(layout=form_input_layout, value=64)
pooling_ratio = FloatText(layout=form_input_layout, value=0.5)
data_file = FileUpload(layout=form_input_layout)
validation_split = FloatText(layout=form_input_layout, value=0.2)
testing_split = FloatText(layout=form_input_layout, value=0.2)
minibatch_size = IntText(layout=form_input_layout, value=10)
ndcg_cutoff = IntText(layout=form_input_layout, value= 10)
submit_button = Button(description='Create Experiment',layout=form_button_layout)

form_rows = [
    widgets.HBox([
        Box([Label(value='Experiment Name'), name], layout=form_item_layout),
    ],layout=form_row_layout),
    widgets.HBox([
        Box([Label(value='Learning Rate'), lr], layout=form_item_layout),
        Box([Label(value='Weight Decay'), weight_decay], layout=form_item_layout),
        Box([Label(value='Dropout Probability'), dropout], layout=form_item_layout),
        Box([Label(value='Leaky ReLU Coefficient'), leaky_relu_coefficient], layout=form_item_layout),
    ],layout=form_row_layout),
    widgets.HBox([
        Box([Label(value='Number of Convolutional Blocks'), num_blocks], layout=form_item_layout),
        Box([Label(value='Number of Feature Channels'), num_channels], layout=form_item_layout),
        Box([Label(value='Graph Pooling Ratio'), pooling_ratio], layout=form_item_layout),
    ],layout=form_row_layout),
    widgets.HBox([
        Box([Label(value='File Containing Screened Compounds'), data_file], layout = large_form_item_layout),
        Box([Label(value='Validation Split'), validation_split], layout=form_item_layout),
        Box([Label(value='Testing Split'), testing_split], layout=form_item_layout),
        Box([Label(value='Minibatch Size'), minibatch_size], layout=form_item_layout),
        Box([Label(value='NDCG Cutoff'), ndcg_cutoff], layout=form_item_layout),
    ],layout=form_row_layout),
    widgets.HBox([submit_button])
]

submit_form = VBox(form_rows)
#, layout=form_item_layout)
display(submit_form)

def create_experiment(button):
    global agent
    model_params = {
        "learning_rate":lr.value,
        "weight_decay":weight_decay.value,
        "dropout":dropout.value,
        "num_blocks":num_blocks.value,
        "num_channels":num_channels.value,
        "pooling_type":"SAG",
        "pooling_ratio":pooling_ratio.value,
        "validation_split":validation_split.value,
        "testing_split":testing_split.value,
        "minibatch_size":minibatch_size.value,
        "ndcg_cutoff":ndcg_cutoff.value
    }
    agent = Agent.create(name.value,model_params,list(data_file.value.values())[0]['content'])
    agent.train(250)
submit_button.on_click(create_experiment)

## Open an Existing Experiment

In [4]:
experiment = Text(layout=form_input_layout)
load_button = Button(description='Load Experiment')
load_form = widgets.VBox([Box([Label(value='Experiment Name'), experiment]),Box([load_button])])

display(load_form)

def load_experiment(button):
    global agent
    agent = Agent(experiment.value)
    summary, points = agent.evaluate()
    plot_results(plot_widget,points)
    summarize_results(summary_widget,summary)

load_button.on_click(load_experiment)

## Training Results

In [5]:
def plot_results(output,points):
    with output:
        clear_output()
        
        fig, ax = plt.subplots(constrained_layout=True)

        ax.set_xlabel('Predicted Ranking')
        ax.set_ylabel('Actual Score')

        training_data = points.loc[points['dataset']=='training']
        validation_data = points.loc[points['dataset']=='validation']
        testing_data = points.loc[points['dataset']=='testing']

        colors = {'training':'blue','testing':'green','validation':'orange'}

        training_scatter = ax.scatter(training_data.x,training_data.y,c=colors['training'])
        validation_scatter = ax.scatter(validation_data.x,validation_data.y,c=colors['validation'])
        testing_scatter = ax.scatter(testing_data.x,testing_data.y,c=colors['testing'])

        dmso_score = ax.axhline(y=4.276,c='black',ls='dashed')
        ndcg_cutoff = ax.axvline(x=9.5,c='black',ls='dotted')

        ax.legend([training_scatter,validation_scatter,testing_scatter,dmso_score,ndcg_cutoff],['Training Data','Validation Data','Testing Data','DMSO Score','NDCG Cutoff'])

        plt.show()

In [6]:
def summarize_results(output,summary):
    clear_output()
    
    with output:
        df = pd.DataFrame(summary)
        display(df)

In [7]:
plot_widget = Output()
summary_widget = Output()
results = HBox([plot_widget,summary_widget])
display(results)

## Conduct a Virtual Screen

#### To virtually screen a set of compounds, create a CSV file that contains the following columns

- **SMILES:** The cannonical SMILES string for a given compound

#### An example of such a file is shown below

|Name|SMILES|
|---|---|
|Dopamine|C1=CC(=C(C=C1CCN)O)O|
|Serotonin|C1=CC2=C(C=C1O)C(=CN2)CCN|
|Epinephrine|CNCC(C1=CC(=C(C=C1)O)O)O|
|Norepinephrine|C1=CC(=C(C=C1C(CN)O)O)O|

#### Once you've compiled the file, press the upload button to screen your selected compounds

In [8]:
def screen(file):
    if agent != None and file != {}:
        df = agent.screen(file)
        display(HTML(df.to_html(escape=False)))
        
prediction = interact(screen,file=FileUpload())

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…